In [1]:
using TensorFlow
using Distributions
using MLDataUtils
using SwiftObjectStores
using Iterators
using ColoringNames
using MLLabelUtils
using StaticArrays
using Base.Test
using ProgressMeter
using Memoize
using StatsBase

In [2]:
const serv=SwiftService()
#const train_raw = get_file(fh->readdlm(fh,'\t'), serv, "color", "monroe/train.csv")
const valid_raw = get_file(fh->readdlm(fh,'\t'), serv, "color", "monroe/dev.csv")

108545×4 Array{Any,2}:
 "acid green"  0.328092   0.665272  0.937255
 "acid green"  0.241007   0.712821  0.764706
 "acid green"  0.274707   0.780392  1.0     
 "acid green"  0.353462   0.896104  0.905882
 "acid green"  0.223856   0.846473  0.945098
 "acid green"  0.219101   0.700787  0.996078
 "acid green"  0.20546    0.954733  0.952941
 "acid green"  0.184783   0.987124  0.913725
 "acid green"  0.204225   0.883817  0.945098
 "acid green"  0.253876   0.830918  0.811765
 "acid green"  0.299065   0.968326  0.866667
 "acid green"  0.298712   0.936652  0.866667
 "acid green"  0.287597   0.968468  0.870588
 ⋮                                          
 "yuck"        0.142694   0.67907   0.843137
 "yuck"        0.146505   0.751515  0.647059
 "yuck"        0.133333   0.9       0.392157
 "yuck"        0.212598   0.557018  0.894118
 "yuck"        0.224886   0.672811  0.85098 
 "yuck"        0.246124   0.597222  0.564706
 "yuck"        0.193157   0.782383  0.756863
 "yuck"        0.0766284  0.4627

In [3]:
const tokenize = morpheme_tokenizer("../data/replacement_rules.csv")
@memoize demarcate(tokens, starter="<S>", ender="</S>") = [starter; tokens; ender]

@test demarcate(["a", "b", "c"]) == ["<S>", "a", "b", "c", "</S>"]

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in symbol(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in @memoize(::Expr, ::Vararg{Expr,N}) at /home/uniwa/students2/students/20361362/linux/.julia/v0.5/Memoize/src/Memoize.jl:14
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/uniwa/students2/students/20361362/linux/.julia/v0.5/IJulia/src/execute_request.jl:157
 in eventloop(::ZMQ.Socket) at /home/uniwa/students2/students/20361362/linux/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##13#19)() at ./task.jl:360
while loading In[3], in expression starting on line 2


Test Passed
  Expression: demarcate(["a","b","c"]) == ["<S>","a","b","c","</S>"]
   Evaluated: String["<S>","a","b","c","</S>"] == String["<S>","a","b","c","</S>"]

In [4]:
const od =(MLDataUtils.ObsDim.First(), MLDataUtils.ObsDim.Last())

(MLDataUtils.ObsDim.Constant{1}(),MLDataUtils.ObsDim.Last())

In [5]:
function prepare_data(raw, encoding_=nothing)
    labels = convert(Vector{String}, raw[:,1]);
    hsv_data = convert(Matrix{Float64}, raw[:,2:end]);
    tokenized_labels = demarcate.(tokenize.(labels))
    local encoding
    if encoding_===nothing
        all_tokens = reduce(union, tokenized_labels)
        encoding = labelenc(all_tokens)
    else
        encoding = encoding_
    end
    
    label_inds = map(toks->label2ind.(toks, Scalar(encoding)), tokenized_labels)
    rpad_to_matrix(label_inds), hsv_data, encoding
end


prepare_data (generic function with 2 methods)

In [6]:
const valid_terms_padded, valid_hsv, encoding = prepare_data(valid_raw)
#const train_terms_padded, train_hsv,  encoding = prepare_data(train_raw, encoding)

(
[1 1 … 1 1; 2 2 … 313 313; … ; 0 0 … 0 0; 0 0 … 0 0],

[0.328092 0.665272 0.937255; 0.241007 0.712821 0.764706; … ; 0.0540541 0.308333 0.941176; 0.125874 0.590909 0.94902],

MLLabelUtils.LabelEnc.NativeLabels{String,313}(String["<S>","acid","green","</S>","adobe","algae","almost","black","amber","amethyst"  …  "vivid","vomit","warm","watermelon","weird","wheat","wine","wintergreen","wisteria","yuck"],Dict("russet"=>257,"celery"=>88,"lipstick"=>201,"dull"=>141,"chocolate"=>95,"sepia"=>265,"fern"=>154,"sickly"=>269,"y"=>46,"heliotrope"=>175…)))

In [7]:
get_mask(V, dtype=Float32)=cast(V, Bool)
apply_mask(V, mask) = gather_nd(V, find(mask))
#V.*tile(expand_dims(mask, 1), [1, get_shape(V,2)])

apply_mask (generic function with 1 method)

In [8]:
#DEFINITION
function color_to_terms_network(n_classes, n_steps;
        batch_size = 128,
        learning_rate = 0.05,
        hidden_layer_size = 256,
        embedding_dim = 16
    )
    n_input = 3 # HSV
    
        
    ###################################
    sess = Session(Graph())
    tensor_vars = @names_from begin
        X_hsv = placeholder(Float32, shape=[batch_size, n_input]; name="X_HSVs")
        Term_obs_s = placeholder(Int32, shape=[n_steps+1, batch_size]; name="Term_obs_s")
        
        EmbeddingTable = get_variable("TokenEmbeddings3",  [n_classes, embedding_dim], Float32; initializer=Normal(0, .1))

        
        #Mangle Terms into shape
        Term_obs_s_out = slice(Term_obs_s, [1,0], [-1,-1]) #Don't want first input "<S>" #0based
        TT = reshape(Term_obs_s_out, [n_steps*batch_size]; name="Stack_Term_obs_s")
        
        Term_obs_s_ins = unpack(Term_obs_s+1)[1:end-1]#Don't want last input "</S>" (or padding character often but we will handle that seperately)
        Tes = gather.(Scalar(EmbeddingTable), Term_obs_s_ins)
        
        @show get_shape.(Tes)


        #Mangle colors into shape
        X_h, X_s, X_v = unpack(X_hsv; axis=2)
        X_h = reshape(X_h, [batch_size])     
        X_hr = X_h.*2π
        X_col = pack((sin(X_hr), cos(X_hr), X_s-0.5, X_v-0.5); axis=2) #Smooth hue by breaking into cos and sin, and zero mean everything else1       
        Xs = [concat(2, [X_col, T]; name="Xs$ii") for (ii,T) in enumerate(Tes)]#Pair color input at each step with previous term
        

        Hs, states = nn.rnn(nn.rnn_cell.LSTMCell(hidden_layer_size), Xs; dtype=Float32)#, sequence_length=n_steps);
        W1 = get_variable("weights2", [hidden_layer_size, n_classes], Float32;  initializer=Normal(0, .1))
        B1 = get_variable("bias2", [n_classes], Float32;  initializer=Normal(0, .1))
        Ls =  [H*W1+B1 for H in Hs]


        LL = concat(1, Ls; name="Stack_Logits")

        mask = get_mask(TT)
        TT_masked = apply_mask(TT, mask)
        LL_masked = apply_mask(LL, mask)
        @show get_shape(find(mask))
        costs = nn.sparse_softmax_cross_entropy_with_logits(LL_masked, TT_masked+1)
        cost = reduce_mean(-costs) #cross entropy
        optimizer = train.minimize(train.AdamOptimizer(learning_rate), cost)
        
        
        Term_preds_onehots = nn.softmax(LL; name="Term_preds_onehots")
        Term_preds_s = reshape(indmax(Term_preds_onehots, 2)+1, [n_steps, batch_size]) #TODO: this messes up zero entries, not that it matters

        #costs  = reduce_sum(Term_obs_onehots.*Term_preds_onehots_log, reduction_indices=[1])
        
        
    end
    ########## GET it running
    
    run(sess, initialize_all_variables())
    
    return sess, tensor_vars 
end

color_to_terms_network (generic function with 1 method)

In [9]:
const batch_size = 128*4
n_steps=size(valid_terms_padded,1)-1
n_classes = nlabel(encoding)+1
#sess, t = 
et=color_to_terms_network(n_classes, n_steps;
        learning_rate = 0.05,
        hidden_layer_size = 32,
        embedding_dim = 16,
        batch_size=batch_size
    )

get_shape.(Tes) = TensorFlow.ShapeInference.TensorShape[TensorShape[512, 16],TensorShape[512, 16],TensorShape[512, 16],TensorShape[512, 16],TensorShape[512, 16]]
get_shape(find(mask)) = 

2017-02-23 14:58:43.660849: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-02-23 14:58:43.660880: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-02-23 14:58:43.660884: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2017-02-23 14:58:43.660887: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.


TensorShape[?]


LoadError: On worker 2:
Python error: Input 'shape' of 'ScatterNd' Op has type int32 that does not match type int64 of argument 'indices'.
 in py_gradients at /home/uniwa/students2/students/20361362/linux/.julia/v0.5/TensorFlow/src/py.jl:38
 in #17 at /home/uniwa/students2/students/20361362/linux/.julia/v0.5/TensorFlow/src/core.jl:1047
 in #625 at ./multi.jl:1421
 in run_work_thunk at ./multi.jl:1001
 in macro expansion at ./multi.jl:1421 [inlined]
 in #624 at ./event.jl:68

In [ ]:
function train_from_terms!(sess, t::Associative{Symbol}, train_terms_padded, train_hsv; epochs=3)
    batchs = eachbatch(shuffleobs((train_hsv, train_terms_padded); obsdim=od); size=batch_size, obsdim=od)
    local cost_o
    for ii in 1:epochs
        @show ii
        @showprogress for (hsv,terms) in batchs
            Term_preds_s_o, TT_s_o, cost_o, optimizer_o = run(sess,
                [t[:Term_preds_s], t[:TT], t[:cost], t[:optimizer]], 
            Dict(t[:X_hsv]=>hsv, t[:Term_obs_s]=>terms))
        end
    end
    @show cost_o
end
#obs,pred, oo, pp = run(sess, [Term_obs_onehots, Term_preds_onehots, Term_obs_s_out, Term_preds_s], Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels))
train_from_terms!(sess, t, train_terms_padded, train_hsv; epochs=3)

In [ ]:
"A quick hack to calculate accuracy"
function masked_acc(obs, pred)
    obs = obs[2:end,:] #skip start token
    @assert(size(obs)==size(pred), "$(size(obs)) != $(size(pred))")
    mpred = sign(obs).*pred
    mean(pp==oo for (pp,oo) in eachobs((mpred, obs)))
end

In [ ]:
"A quick hack to calculate perplexity"
function masked_perp(obs, preds_onehots_log)
    obs = obs[2:end,:] #skip start token
    pred_lp=reshape(preds_onehots_log, (batch_size, n_steps, n_classes))
    expon = 0.0
    for (o_indexes, log_probs) in eachobs((obs, pred_lp),
        obsdim=(MLDataUtils.ObsDim.Last(), MLDataUtils.ObsDim.First()))
        content_o_indexes = rstrip(o_indexes)[end-1] #strip padding and end token
        expon += sum(log_probs[step, ii] for (step,ii) in enumerate(content_o_indexes))
        # Each probability is conditional on all of the ealier. 
        # Their product is the competely correct (*without* Markov assumption)
        # probability estimate for the whole
        # Adding the Logs is thus a correct log-prob for the whole
    end
    exp(-expon/batch_size)
end

In [ ]:
"""
This evaluation function does not take into account the non-knowing of term t-1.
This is legal for calculating perplexity
"""
function rough_evalute(sess, t::Associative{Symbol}, test_terms_padded, test_hsv)
    batchs = eachbatch((test_hsv, test_terms_padded); size=batch_size, obsdim=od)
    cost_total = 0.0
    acc_total = 0.0
    perps = Float64[]
    preds_o = Matrix{Float64}(size(test_terms_padded,1)-1,0)
    @showprogress for (hsv,terms) in batchs
        preds, preds_lp, cost_o = run(sess,
        [t[:Term_preds_s], t[:Term_preds_onehots_log], t[:cost]], 
        Dict(t[:X_hsv]=>hsv, t[:Term_obs_s]=>terms))
        
        preds_o = [preds_o preds]
        
        push!(perps, masked_perp(terms, preds_lp))
        acc_total += masked_acc(terms, preds)
        cost_total += cost_o
    end

    (cost_total/length(batchs),
    acc_total/length(batchs),
    geomean(perps),
    preds_o)
end
cost, acc, perp, preds_o = rough_evalute(sess, t, valid_terms_padded, valid_hsv)

In [ ]:
[Pair(a,ind2label(b,encoding)) for (a,b) in sort(reverse.(collect(countmap(train_terms_padded[2,:]))), rev=true)[2:end]]

In [ ]:
collect(enumerate(ind2label.(2:50, encoding)))

In [ ]:
unique_cols = first.(unique(last, enumerate(eachobs(preds_o))))
pls = ind2label.(Int.(preds_o[:,unique_cols]), encoding)'

ols_coded=valid_terms_padded
ols_coded[ols_coded.==0]=1
ols = ind2label.(Int.(ols_coded[2:end, unique_cols]), encoding)'

join(mapslices(x->join(x," ") , [ols fill("->", size(unique_cols,1)) pls], 2), "\n") |> print

In [ ]:
methods(ind2label)

In [ ]:
@testset "masks" begin
    mask_sess = Session(Graph())
    a_val = (
    [2.  5  8  9  2
     3.  6  4  4  9
     4.  7  3  2  4
     1.  4  2  4  4]
    )
    A=constant(a_val)
    M = constant([1, 0, 2, 4])
    @show masked = run(mask_sess, squeeze(find(get_mask(M)), [2]))
    @show masked = run(mask_sess, gather_nd(A, [1 3 4]')) 
    @show masked = run(mask_sess, gather_nd(A, find(get_mask(M)))) 
    
    
    masked = run(mask_sess, apply_mask(A, get_mask(M)))
    
    @test all(masked[1,:] .== a_val[1,:])
    @test all(masked[2:3,:] .== a_val[3:4,:])
end;

In [ ]:
##MASK test code
#TODO 
if test_it 
        @assert isa(run(sess, cost, Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels)), Number )
        ## MASK TEST######################
        ll, ll2 = run(sess, [ LL_masked, 
            reshape(tile(expand_dims(get_mask(Term_obs_s_out),2),[1,1,8]).*concat(0, expand_dims.(Ls, Scalar(0))), [batch_size*n_steps, n_classes])
        ], Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels))
        @assert ll ≈ ll2
    end

In [ ]:
sess = Session(Graph())
x = placeholder(Int64)
run(sess, get(cast(x, Int32)), Dict(x=>2))